In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_md



2023-10-13 16:46:33.705744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00


In [ ]:
!pip install spacy-lookups-data


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.3 MB/s eta 0:00:00


In [ ]:
import spacy
import torch
from transformers import AutoModel, AutoTokenizer
import json
from spacy.training.example import Example

# Load spaCy model for linguistic preprocessing
nlp = spacy.load("en_core_web_md")

# Load BERT model and tokenizer
bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModel.from_pretrained(bert_model_name)

# Load the JSON dataset
with open('training_data.json', 'r') as json_file:
    data = json.load(json_file)

# Define a function to obtain BERT embeddings for a given text
def get_bert_embeddings(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(**tokens)
    return outputs.last_hidden_state

# Process the data and create examples for training
train_data = []

for item in data:
    text = item["text"]
    annotations = item["annotations"]

    doc = nlp.make_doc(text)
    bert_embeddings = get_bert_embeddings(text)

    entities = []

    for ann in annotations:
        start, end, label = ann["start"], ann["end"], ann["label"]
        entities.append((start, end, label))

    example = Example.from_dict(doc, {"entities": entities})
    example.reference.user_data["bert_embeddings"] = bert_embeddings

    train_data.append(example)

# Define a blank spaCy NER model
nlp_ner = spacy.blank("en")

# Add a named entity recognition component to the spaCy pipeline with the correct transition scheme
from spacy.pipeline.ner import BiluoPushDown

ner = nlp_ner.add_pipe("ner", config={"transition_scheme": BiluoPushDown.transition_scheme})

# Add labels to the NER model
for example in train_data:
    for ent in example.reference.ents:
        label = ent.label_
        if label not in ner.labels:
            ner.add_label(label)

# Train the NER model using the combined data
nlp_ner.begin_training()

for example in train_data:
    ner.update([example])

# Save the trained NER model to a file
nlp_ner.to_disk('custom_ner_model')


ImportError: ignored